# Preparación de datos
*"Es un error capital teorizar antes de tener datos. Sin darse cuenta, uno empieza a deformar los hechos para que se ajusten a las teorías, en lugar de ajustar las teorías a los hechos."*  Arthur Conan Doyle en "Escándalo en Bohemia".

## Introducción
El análisis de datos comienza mucho antes de aplicar modelos estadísticos o construir visualizaciones. Dos etapas clave que determinan la calidad del análisis son la preparación de datos y el procesamiento de datos.

La preparación de datos es una etapa inicial con enfoque manual y exploratorio. Aquí el objetivo es comprender la estructura y calidad de los datos: se revisan los tipos de variables, se detectan valores faltantes, se identifican errores o inconsistencias, y se decide qué información conservar. Esta etapa implica el juicio humano y suele apoyarse en comandos como .head(), .info(), .describe(), así como inspección visual directa del contenido.

Una vez comprendidos los datos, pasamos al procesamiento de datos, que implica un enfoque automatizado y reproducible. En esta etapa se aplican transformaciones programadas que pueden incluir la eliminación de duplicados, tratamiento de valores perdidos y tratamiento de valores atípicos. Estas tareas se sistematizan mediante funciones, scripts o pipelines, lo que permite aplicar los mismos pasos a nuevos conjuntos de datos sin intervención manual.

Entender esta distinción ayuda a formar buenos hábitos: primero explorar con criterio, luego automatizar con claridad.

## Descripción de datos
Airbnb es una plataforma en línea que permite a los usuarios alquilar alojamientos. El archivo "listings_cdmx.csv" contiene el listado de alojamientos en la Ciudad de México, con información básica actualizada al 25 de septiembre de 2024. 

Las variables disponibles son: 
- id: identificador del anuncio de alojamiento.
- name: nombre del alojamiento.
- host_id: identificador del anfitrión.
- host_name: nombre del anfitrión.
- neighbourhood_group: grupo.
- neighbourhood: alcaldía.
- latitude: latitud.
- longitude: longitud.
- room_type: puede ser "Entire place", "Private room", "Shared room" u "Hotel".
- price: precio en moneda nacional.
- minimum_nights: mínimo de noches de estadía.
- last_review: fecha de la última reseña.
- reviews_per_month: promedio de reseñas por mes en el tiempo publicado.
- calculated_host_listings_count: número de anuncios que tiene el anfitrión.
- availability_365: días disponibles en los siguientes 365 días.
- number_of_reviews_ltm: número de reseñas en los últimos 12 meses.
- license: número de licencia o registro.

## Carga y selección de datos

In [1]:
# Importar bibliotecas
import pandas as pd
import numpy as np

**Usa de nombres descriptivos para cada DataFrame**: Aunque es común usar `df` como nombre de un DataFrame, es recomendable emplear nombres descriptivos que indiquen si los datos son originales (*df_original, raw_data, input_data, source_df*), seleccionados o procesados. Esto facilitará la comprensión del código.

In [2]:
# Carga de archivo
url = 'https://github.com/adan-rs/AnalisisDatos/raw/main/data/listings_cdmx.csv'
df_original = pd.read_csv(url)

**Selecciona las columnas**: Si de antemano ya sabes con qué columnas vas a trabajar, puedes utilizar el parámetro `usecols` directamente en la función `pd.read_csv` o `pd.read_excel`. Esto tiene la ventaja de optimizar el espacio en memoria que ocupa el dataframe. Por ejemplo:
```
url = 'https://github.com/adan-rs/AnalisisDatos/raw/main/data/listings_cdmx.csv'
columnas = ["host_id", "neighbourhood", "room_type", "price", 
             "minimum_nights", "number_of_reviews_ltm", "license"]
df_original = pd.read_csv(ruta, usecols=columnas)
```
Sin embargo, en un análisis exploratorio es preferible cargar los datos completos y luego seleccionar las columnas como haremos a continuación:

In [3]:
# Selección de columnas
columnas = ["host_id", "neighbourhood", "room_type", "price", 
            "minimum_nights", "number_of_reviews_ltm", "license"]
df_columnas = df_original[columnas]

**Selección de filas**: Para seleccionar filas bajo condiciones específicas, usa expresiones lógicas. Supongamos que sólo nos interesa analizar casas/departamentos que estén hayan tenido una reseña en el último año. Podemos filtrar entonces:
- Anuncios de casas o departamentos:  
  `df_columnas['room_type'] == 'Entire home/apt'`
- Anuncios con al menos una reseña en el último año:  
  `df_columnas['number_of_reviews_ltm'] > 0`

Combinaremos ambas condiciones en un filtro colocando cada condición dentro de un paréntesis y luego aplicamos ese filtro con `df_filtrado = df_filtrado[filtro]` o con `df_filtrado = df_filtrado.loc[filtro]`

In [4]:
# Selección de filas (a partir del dataframe con columnas seleccionadas)
filtro = (df_columnas['room_type'] == 'Entire home/apt') & (df_columnas['number_of_reviews_ltm'] > 0)
df_filtrado = df_columnas[filtro]

Es recomendable utilizar `.copy()` para trabajar con una copia independiente y no una vista ligada al original. Esto es importante porque con esto se evita el mensaje `SettingWithCopyWarning` al hacer modificaciones a este dataframe.

In [5]:
# Crear copia del DataFrame original
df = df_filtrado.copy()

**Encapsulación del proceso (opcional)**: Para mantener un código limpio y reutilizable, encapsula los pasos anteriores en una función personalizada con un nombre descriptivo:

In [6]:
def load_and_filter(filepath):
    """
    Carga y filtrado de datos originales.
    """
    columns = ["host_id", "neighbourhood", "room_type", "price", 
               "minimum_nights", "number_of_reviews_ltm", "license"]
    data = pd.read_csv(filepath, usecols=columns)
    filtro = (data['room_type'] == 'Entire home/apt') & (data['number_of_reviews_ltm'] > 0)
    return data.loc[filtro].copy()

In [7]:
url = 'https://github.com/adan-rs/AnalisisDatos/raw/main/data/listings_cdmx.csv'
df = load_and_filter(url)

## Revisión general de los datos

Revisa la información del DataFrame con los siguientes comandos:
- `df.columns`: muestra los nombres de las columnas 
- `df.shape`: muestra el número de filas y columnas.
- `df.info()`: proporciona un resumen de las columnas, incluyendo sus tipos de datos y valores nulos.
- `df.head()`: muestra las primeras filas del DataFrame.

In [8]:
# Número de filas y columnas
df.shape

(13980, 7)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13980 entries, 2 to 26548
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   host_id                13980 non-null  int64  
 1   neighbourhood          13980 non-null  object 
 2   room_type              13980 non-null  object 
 3   price                  13438 non-null  float64
 4   minimum_nights         13980 non-null  int64  
 5   number_of_reviews_ltm  13980 non-null  int64  
 6   license                0 non-null      float64
dtypes: float64(2), int64(3), object(2)
memory usage: 873.8+ KB


In [10]:
df.head(5)

,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews_ltm,license
2,334451,Cuauhtémoc,Entire home/apt,2029.0,2,4,NaN
3,212109,Coyoacán,Entire home/apt,1140.0,5,7,NaN
6,790208,Miguel Hidalgo,Entire home/apt,3671.0,2,36,NaN
8,36836,Cuauhtémoc,Entire home/apt,1642.0,3,7,NaN
11,899360,Cuauhtémoc,Entire home/apt,11000.0,1,6,NaN


In [11]:
# Tipos de datos
df.dtypes.value_counts()

int64      3
object     2
float64    2
Name: count, dtype: int64

In [12]:
# Valores nulos por variable
df.isnull().sum()

host_id                      0
neighbourhood                0
room_type                    0
price                      542
minimum_nights               0
number_of_reviews_ltm        0
license                  13980
dtype: int64

*Estadística descriptiva: medidas numéricas (opcional)*: Si deseas obtener las principales medidas numéricas de las variables cualitativas utiliza `df.describe()`.

*Estadística descriptiva: variables categóricas (opcional)*: Si deseas conocer el número de valores únicos en cada variable categórica puedes utilizar `df[col].nunique()`.

## Análisis rápido

Construiremos una función para el análisis rápido de los datos.

In [13]:
def quick_analysis(df):
    """Análisis rápido del DataFrame"""
    print("\nANÁLISIS RÁPIDO")
    print("-" * 50)
    print(f"Dimensiones del DataFrame: {df.shape}")
    print(f"Tipos de datos:\n{df.dtypes}")
    print(f"\nValores faltantes:")
    missing = df.isnull().sum()
    print(missing[missing > 0])
    
    print(f"\nMedidas numéricas:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe().round(2))
    
    print(f"\nVariables categóricas:")
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        print(f"{col}: {df[col].nunique()} valores únicos")

En el código anterior se utilizó una *f-string*. Una f-string (formatted string literal) es una forma concisa e intuitiva de incluir variables o expresiones dentro de un texto en Python. Se introdujo en Python 3.6 y su sintaxis es `f"Texto con una variable: {variable}"`.

In [14]:
quick_analysis(df)


ANÁLISIS RÁPIDO
--------------------------------------------------
Dimensiones del DataFrame: (13980, 7)
Tipos de datos:
host_id                    int64
neighbourhood             object
room_type                 object
price                    float64
minimum_nights             int64
number_of_reviews_ltm      int64
license                  float64
dtype: object

Valores faltantes:
price        542
license    13980
dtype: int64

Medidas numéricas:
            host_id      price  minimum_nights  number_of_reviews_ltm  license
count  1.398000e+04   13438.00        13980.00               13980.00      0.0
mean   2.156024e+08    1625.04            3.11                  20.75      NaN
std    1.878794e+08    4027.91           14.40                  21.59      NaN
min    7.365000e+03     169.00            1.00                   1.00      NaN
25%    4.550366e+07     870.00            1.00                   5.00      NaN
50%    1.472542e+08    1200.00            2.00                  14.00   

**Detección de inconsistencias en tipos de variables:** El análisis de la información anterior nos revela que, aunque la columna host_id contiene valores numéricos  no representa una variable cuantitativa. Debemos reclasificarla para evitar operaciones matemáticas inapropiadas:

In [15]:
df["host_id"] = df["host_id"].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13980 entries, 2 to 26548
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   host_id                13980 non-null  object 
 1   neighbourhood          13980 non-null  object 
 2   room_type              13980 non-null  object 
 3   price                  13438 non-null  float64
 4   minimum_nights         13980 non-null  int64  
 5   number_of_reviews_ltm  13980 non-null  int64  
 6   license                0 non-null      float64
dtypes: float64(2), int64(2), object(3)
memory usage: 873.8+ KB


# Procesamiento de datos

## Datos duplicados
**Identificar datos duplicados**: En ocasiones, nuestra base de datos puede contener información duplicada. Para identificar estos datos duplicados, podemos utilizar el método duplicated() de las siguientes maneras:
- Para visualizar todas las filas duplicadas:  
  `df[df.duplicated()]`
- Si nos interesa analizar duplicados en una columna específica (por ejemplo, "A"):  
  `df[df['A'].duplicated()]`

In [16]:
# Conteo de duplicados en todas las variables 
numero_duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {numero_duplicados}")

Filas duplicadas: 119


**Eliminar datos duplicados**: Para eliminar datos duplicados, se utiliza el método `drop_duplicates()` que conserva la primera ocurrencia de cada duplicado. En el ejemplo general, podemos eliminar filas duplicadas (considerando todas las variables) mediante:  
`df = df.drop_duplicates()`  
Si necesitamos eliminar duplicados considerando sólo algunas variables, podemos usar el parámetro subset:  
`df = df.drop_duplicates(subset=['hotel_id'])`

In [17]:
df = df.drop_duplicates(df)

## Datos perdidos
Los datos perdidos pueden influir significativamente en los resultados del análisis, por lo que es importante determinar si estos son aleatorios o si siguen algún patrón específico. Por ejemplo, en una encuesta, es posible que quienes omitan informar su ingreso compartan ciertas características o que preguntas sobre comportamientos socialmente indeseables tengan tasas de no respuesta más altas.

Para determinar si los datos perdidos son aleatorios, es común realizar pruebas estadísticas. El procedimiento general incluye:
- Dividir las observaciones en dos grupos:
    - Grupo con datos completos.
    - Grupo con valores perdidos en una variable específica.
- Comparar los valores promedio de otras variables entre ambos grupos mediante pruebas de significancia.

Estas pruebas permiten identificar si los datos perdidos podrían estar relacionados con alguna característica específica de los datos.

**Análisis inicial de datos perdidos**: Una herramienta útil para identificar variables con valores perdidos es el método `info()`:

In [18]:
# Identifica qué variables tienen valores perdidos con 'info()'
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13861 entries, 2 to 26548
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   host_id                13861 non-null  object 
 1   neighbourhood          13861 non-null  object 
 2   room_type              13861 non-null  object 
 3   price                  13322 non-null  float64
 4   minimum_nights         13861 non-null  int64  
 5   number_of_reviews_ltm  13861 non-null  int64  
 6   license                0 non-null      float64
dtypes: float64(2), int64(2), object(3)
memory usage: 866.3+ KB


Para identificar filas con al menos un valor perdido, podemos usar `df[df.isna().any(axis=1)]`
donde`isna()` identifica los valores perdidos y `any(axis=1)` verifica si existe al menos un valor perdido en cada fila.

Para encontrar las filas con datos perdidos de una variable en particular (por ejemplo "price") podemos usar `df[df.price.isna()]`. Otra opción equivalente a `isna()` es `isnull()`.

In [19]:
# Identificar filas con datos perdidos en una variable
df[df.price.isna()]

,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews_ltm,license
76,4344353,Coyoacán,Entire home/apt,NaN,2,1,NaN
91,13007874,Coyoacán,Entire home/apt,NaN,3,2,NaN
93,12774111,Coyoacán,Entire home/apt,NaN,3,3,NaN
99,5507751,Coyoacán,Entire home/apt,NaN,2,9,NaN
292,18242899,Benito Juárez,Entire home/apt,NaN,1,1,NaN
...,...,...,...,...,...,...,...
25297,246074764,Coyoacán,Entire home/apt,NaN,2,8,NaN
25327,255503292,Miguel Hidalgo,Entire home/apt,NaN,1,8,NaN
25637,594004124,Cuauhtémoc,Entire home/apt,NaN,1,1,NaN
25695,529144733,Venustiano Carranza,Entire home/apt,NaN,2,1,NaN


**¿Qué hacer con datos perdidos?**  
Existen diversas estrategias para tratar los datos faltantes, dependiendo del contexto y las características del problema. A continuación, se presentan las más comunes:

1. *Eliminar filas o columnas con datos faltantes*: Si una columna no es esencial y tiene una gran cantidad de datos faltantes (por ejemplo, más del 20%), puede ser razonable excluirla del análisis. Si solo unas pocas filas contienen datos faltantes (por ejemplo, menos del 5%) y estas ausencias parecen ser completamente aleatorias, eliminarlas puede ser una solución viable. Sin embargo, en el caso de series de tiempo, eliminar filas puede distorsionar los patrones temporales y no es una práctica recomendada (¿por qué crees que esto ocurre?).

2. *Codificación*: Los valores faltantes también pueden contener información valiosa. En variables categóricas, se pueden codificar como una categoría adicional para incluirlos en el análisis.

3. *Imputación simple*: Consiste en reemplazar los valores faltantes con un estimado calculado a partir de la misma variable. Una opción común y conservadora es usar el promedio o la mediana. Sin embargo, este método tiene limitaciones, ya que reduce la variabilidad y puede sesgar los intervalos de confianza (Treiman, 2009).

4. *Imputación multivariada*: Este enfoque utiliza otras variables del conjunto de datos para estimar los valores faltantes. Por ejemplo, en la imputación basada en regresión, se construye una ecuación de regresión donde la variable dependiente es aquella con datos faltantes. Luego, los valores se predicen utilizando la ecuación estimada.

Para profundizar en otros métodos de imputación, se recomienda la lectura del capítulo “Multiple Imputation of Missing Data” de Treiman (2009).

*Imputación simple o univariada*  
La imputación con la media para una columna 'A' se obtiene con:   
`df['A'] = df['A'].fillna(df['A'].mean())`  
En caso de querer la imputación con la mediana se reemplaza `mean()` con `median()` y en el caso de la moda se reemplaza con `mode().iloc[0]` 

In [20]:
def handle_missing_data(df, threshold=0.2):
    """ 
    Maneja datos perdidos según el tipo de datos de cada columna, 
    y elimina columnas con un porcentaje de datos perdidos mayor a 
    un umbral específico. 
    Retorna DataFrame procesado sin valores nulos.
    """  
    print("\nTRATAMIENTO DE VALORES PERDIDOS")
    print("-" * 50)
    print("Antes: valores faltantes por columna:")
    print(df.isnull().sum())
    
    # Remove columns with high proportion of missing values
    cols_with_nulls = df.isnull().mean() > threshold
    cols_to_drop = df.columns[cols_with_nulls]
    if len(cols_to_drop) > 0:
        print(f"\nColumnas eliminadas por tener más de {threshold*100}% de valores nulos:")
        print(cols_to_drop.tolist())
    df = df.drop(columns=cols_to_drop)
    
    # Handle missing values in remaining columns usando loc (evita el warning)
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            # Para columnas numéricas: usar mediana
            mask = df[col].isna()
            df.loc[mask, col] = df[col].median()
        elif df[col].dtype == 'object':
            # Para columnas categóricas: usar 'DESCONOCIDO'
            mask = df[col].isna()
            df.loc[mask, col] = 'DESCONOCIDO'
            
    total_remaining_nulls = df.isna().sum().sum()
    print(f"\nDespués: valores nulos restantes: {total_remaining_nulls}")
    
    return df

En el código anterior aparece un *iterador* `for..in` que se usa para recorrer de manera secuencial los elementos de una colección

In [21]:
df = handle_missing_data(df)


TRATAMIENTO DE VALORES PERDIDOS
--------------------------------------------------
Antes: valores faltantes por columna:
host_id                      0
neighbourhood                0
room_type                    0
price                      539
minimum_nights               0
number_of_reviews_ltm        0
license                  13861
dtype: int64

Columnas eliminadas por tener más de 20.0% de valores nulos:
['license']

Después: valores nulos restantes: 0


**Nota**: Estas funciones están diseñadas para datos de corte transversal (cross-sectional) y no deben aplicarse directamente a series de tiempo. En series de tiempo hay que considerar los patrones que tenga la serie para hacer la estimación o interpolación apropiada.

## Valores atípicos
Un valor atípico (outlier) es un dato extremo en una o más variables que se desvía significativamente del resto de las observaciones. 

**Identificación de datos atípicos**  
- En el caso de series univariadas, es común utilizar el método del valor z o el criterio del rango intercuartil para identificar valores atípicos.
- Para el caso de datos multivariados se utilizan técnicas estadísticas avanzadas o algoritmos como *isolation forest*.

**¿Qué hacer con datos atípicos?**
- Si el valor extremo es un error de captura o parte de otra población lo recomendable es corregir o borrar el caso o variable.
- Si el valor extremo es parte de los datos que nos interesa analizar se debe mantener (p. ej. ventas en navidad).
- En algunas variables económicas se recomienda transformar la variable de forma tal que el valor extremo no impacte los resultados (p. ej. transformación logarítmica, transformación de Box-Cox o la recodificación de datos).


*Método del valor z*: En este enfoque un valor atípico es aquel que esté a más de tres desviaciones estándar a partir de la media. Es un enfoque sencillo y ampliamente utilizado en series univariadas.

In [22]:
def remove_outliers_3s(df, column):
    """
    Elimina valores atípicos utilizando el método del valor z. 
    Retorna un DataFrame sin valores atípicos.
    """
    mean = df[column].mean()
    std = df[column].std()
    upper_limit = mean + 3 * std
    lower_limit = mean - 3 * std
    df_clean = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    excluded_values = len(df) - len(df_clean)
    print(f"Cantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [23]:
remove_outliers_3s(df,'price')

Cantidad de valores atípicos excluidos: 42


,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews_ltm
2,334451,Cuauhtémoc,Entire home/apt,2029.0,2,4
3,212109,Coyoacán,Entire home/apt,1140.0,5,7
6,790208,Miguel Hidalgo,Entire home/apt,3671.0,2,36
8,36836,Cuauhtémoc,Entire home/apt,1642.0,3,7
11,899360,Cuauhtémoc,Entire home/apt,11000.0,1,6
...,...,...,...,...,...,...
26491,541272314,Miguel Hidalgo,Entire home/apt,2639.0,1,1
26501,71028639,Cuauhtémoc,Entire home/apt,899.0,1,2
26536,559292840,Cuauhtémoc,Entire home/apt,1200.0,1,1
26541,333035396,Cuauhtémoc,Entire home/apt,1360.0,1,1


*Método del rango intercuartil*. Otro criterio común es considerar como atípicos los valores que están a más de 1.5 veces el rango intercuartil hacia el extremo a partir del 1er. o 3er. cuartil.

In [24]:
def remove_outliers_iqr(df, column):
    """
    Elimina valores atípicos utilizando el criterio del rango intercuartil.
    Retorna DataFrame sin valores atípicos.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    df_clean = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    excluded_values = len(df) - len(df_clean)
    print(f"Cantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [25]:
remove_outliers_iqr(df, 'price')

Cantidad de valores atípicos excluidos: 1033


,host_id,neighbourhood,room_type,price,minimum_nights,number_of_reviews_ltm
2,334451,Cuauhtémoc,Entire home/apt,2029.0,2,4
3,212109,Coyoacán,Entire home/apt,1140.0,5,7
8,36836,Cuauhtémoc,Entire home/apt,1642.0,3,7
12,970609,Cuauhtémoc,Entire home/apt,800.0,2,20
13,998053,Cuauhtémoc,Entire home/apt,780.0,3,1
...,...,...,...,...,...,...
26491,541272314,Miguel Hidalgo,Entire home/apt,2639.0,1,1
26501,71028639,Cuauhtémoc,Entire home/apt,899.0,1,2
26536,559292840,Cuauhtémoc,Entire home/apt,1200.0,1,1
26541,333035396,Cuauhtémoc,Entire home/apt,1360.0,1,1


*Algoritmo isolation forest*: Es un algoritmo utilizado para detectar valores atípicos en grandes conjuntos de datos. Divide los datos de manera aleatoria y cuanto más rápido se aísla un punto mayor es la probabilidad de que sea un valor atípico (anomalía)

In [26]:
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    print("\nTRATAMIENTO DE DATOS ATÍPICOS")
    print("-" * 50)
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"Después: valores atípicos excluidos: {excluded_values}")
    return df_clean

Seleccionaremos este último criterio para filtrar datos atípicos

In [27]:
df = remove_outliers_iso_forest(df, columns=['price', 'minimum_nights'])


TRATAMIENTO DE DATOS ATÍPICOS
--------------------------------------------------
Después: valores atípicos excluidos: 693


**Nota**: Estas funciones están diseñadas para datos de corte transversal (cross-sectional) y no deben aplicarse directamente a series de tiempo. En el caso de las series de tiempo, se recomienda utilizar una ventana deslizante (rolling window) y, en lugar de eliminar datos, optar por métodos de estimación o interpolación.

## Exportar datos (opcional)

Un dataframe lo podemos guardar con `df.to_excel('archivo.xlsx', index=False)`

In [28]:
# Exporta el dataframe depurado con el nombre 'output'
# df.to_excel('output.xlsx', index=False)

## Resumen del flujo de trabajo

In [29]:
# Archivo de origen
url = 'https://github.com/adan-rs/AnalisisDatos/raw/main/data/listings_cdmx.csv'

# Selección de datos
df = load_and_filter(url)

# Correcciones
df["host_id"] = df["host_id"].astype('object')
    
# Eliminar filas duplicadas
df = df.drop_duplicates(df)
    
# Manejar datos perdidos
df = handle_missing_data(df)

# Eliminar valores atípicos
df = remove_outliers_iso_forest(df, columns=['price', 'minimum_nights'])


TRATAMIENTO DE VALORES PERDIDOS
--------------------------------------------------
Antes: valores faltantes por columna:
host_id                      0
neighbourhood                0
room_type                    0
price                      539
minimum_nights               0
number_of_reviews_ltm        0
license                  13861
dtype: int64

Columnas eliminadas por tener más de 20.0% de valores nulos:
['license']

Después: valores nulos restantes: 0

TRATAMIENTO DE DATOS ATÍPICOS
--------------------------------------------------
Después: valores atípicos excluidos: 693


## Ejercicio
En un notebook en blanco, ordena el código con las bibliotecas importadas, las funciones creadas y el flujo de trabajo para procesar los datos de Airbnb en otra ciudad. Reemplaza el filepath/url por `'../data/listings_madrid.csv'` o el enlace (url) al archivo original por:  
`'https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2024-09-11/visualisations/listings.csv'`

## Referencias
- Una discusión interesante sobre el tratamiento de datos se puede encontrar en: Treiman, D. J. (2009). *Quantitative data analysis. Doing social research to test ideas*. San Francisco, CA: Jossey-Bass.
- La base de datos fue tomada de https://insideairbnb.com/get-the-data/ para fines no comerciales.